In [20]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
import ast 
from datetime import datetime as dt
import numpy as np

df_trump=pd.read_csv("tweets_donald_trump.csv",sep=",")
df_biden=pd.read_csv("tweets_joe_biden.csv",sep=",")
df_trump=df_trump.dropna(thresh=2)#rimuove le righe con all'interno almeno 2 NA
df_biden=df_biden.dropna(thresh=2)#rimuove le righe con all'interno almeno 2 NA

def clean_tweet(tweet):
  """"sostituisce con uno spazio vuoto tramite regular esxpression i pattern all'interno delle parentesi"""
  return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t]) |(\w+:\/\/\S+)", " ", tweet).split()) 

def prepr(df):
  """ 
  funzione preprocessing dati
  """
  
  df=df[df['text']!="text"]
  df['text'] = df['text'].apply(clean_tweet)
  df.drop(df.columns[7:], axis=1, inplace=True)
  df=df.loc[2:]
  df=df.dropna(thresh=3)
  df=df[df['sentiment'].str.startswith('{')]
  df['sentiment']=df['sentiment'].apply(ast.literal_eval)
  df=pd.concat([df.drop(['sentiment'], axis=1), df['sentiment'].apply(pd.Series)], axis=1)
  
  return df 
    
df_trump=prepr(df_trump)
df_biden=prepr(df_biden)


/tmp/ipykernel_76468/1836691906.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df['text'].apply(clean_tweet)
/tmp/ipykernel_76468/1836691906.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(df.columns[7:], axis=1, inplace=True)
/tmp/ipykernel_76468/1836691906.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vi

In [21]:
df_biden['candidato'] = 'biden'
df_trump['candidato'] = 'trump'


In [22]:
a = pd.DataFrame(df_biden.groupby('sign').size()/df_biden['sign'].count()*100)



In [23]:
a

,0
sign,
negative,17.125984
neutral,38.976378
positive,43.897638


In [24]:
# aggiunta colonna con percentuali tweet negativi, positivi, neutrali  trump
a['trump']=df_trump.groupby('sign').size()/df_trump['sign'].count()*100


In [25]:
a

,0,trump
sign,,
negative,17.125984,14.763780
neutral,38.976378,42.913386
positive,43.897638,42.322835


In [26]:
a.columns=['biden','trump']
a.to_csv('data_for_pie.csv')#salvo i dati in un csv che utilizzerò per i grafici


In [27]:
a

,biden,trump
sign,,
negative,17.125984,14.763780
neutral,38.976378,42.913386
positive,43.897638,42.322835


In [28]:
#concateno i dataframe dei tweet di biden e di trump
df=pd.concat([df_trump, df_biden], ignore_index=True)

In [29]:
df

,text,name,location,verified,retweet,date tweet,sign,polarity,candidato
0,RT BREAKING Massive bombshell hits Donald Trum...,bunnypie,NaN,False,16278,NaN,neutral,0.000000,trump
1,RT BREAKING The Senate Judiciary Committee wil...,Kelly David Dodson,Florida,False,3664,NaN,neutral,0.000000,trump
2,Donald Trump privately admitted he'd lost 2020...,LadyGhost,NaN,False,0,NaN,neutral,0.000000,trump
3,A family member blames the man's attack which ...,Gee,Southern California,False,0,NaN,negative,-0.075000,trump
4,No where safe from FBI and CIA political spyin...,🇺🇸2022=🇩🇪1933 Liberty 6% 💎,"Sugar Land, TX",False,0,NaN,positive,0.228571,trump
...,...,...,...,...,...,...,...,...,...
1011,RT Joe Biden Gas prices are down $1.30. Realit...,D Matheson,Australia,False,279,2022-09-13 20:42:54+00:00,negative,-0.155556,biden
1012,RT _Jordan -Beef -Poultry -Eggs -Milk -Fruit H...,〽️att Leonarddd,NaN,False,807,2022-09-13 20:42:54+00:00,positive,0.500000,biden
1013,RT BREAKING Joe Biden will be permitted to tak...,JT🌊💙☮,"North Carolina, USA",False,2517,2022-09-13 20:42:54+00:00,neutral,0.000000,biden
1014,RT It's astounding that Joe Biden's only plan ...,Scott Alan,NaN,False,705,2022-09-13 20:42:53+00:00,positive,0.300000,biden


In [30]:
#converto la data, non ho bisogno dell'orario
df['new_date_column'] = pd.to_datetime(df['date tweet'],errors='coerce').dt.date

In [31]:
#raggruppo per data e candidato
g = df.groupby(["candidato", "new_date_column"])

In [32]:
#media giornaliera di polarity giornaliera per candidato
daily_averages = g.aggregate({"polarity":np.mean})

In [33]:
daily_averages

,,polarity
candidato,new_date_column,
biden,2022-09-13,0.097552
trump,2022-09-13,0.031397


In [34]:
#converto in un dataframe
daily_averages=pd.DataFrame(daily_averages)
daily_averages.to_csv('media_polarity.csv', index = True)

In [35]:
df=pd.read_csv("media_polarity.csv")


In [36]:
#rimodello il dataset creato, l'indice è la data, le colonne sono i nomi dei candidati e all'interno delle celle troviamo la media giornaliera di polarity
df=df.pivot(index='new_date_column', columns='candidato', values='polarity')

In [37]:
#visualizzo media polarity
df.to_csv('media_polarity.csv')
